# Simulating Online Experiments

## Things I'll Show

- The difference between the normal and binomial distributions
- How the binmomial distirbution changes according to the population proportion and sample size
- The visual representation of significance level and it's meaning
- Difference between a one and two-sided hypothesis test
- Simulated experiment where there is no difference between the control and exposed population conversion rates
  - results depend on sample size (power)

## Plan

My objective is to show why it's bad science to calculate a p-value for an experiment multiple times. To do that I'm going to try to explain each of the components of the calculation, from the beginning.

### The statistics 



In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, binom
from ipywidgets import interact, FloatSlider, IntSlider, fixed, RadioButtons

# Set the Seaborn style
sns.set(style="dark")

## Normal Distribution

In [2]:
def plot_normal_distribution(mean, variance):
    x = norm.rvs(loc=mean, scale=variance, size=1000, random_state=None)
    y = norm.pdf(x, mean, np.sqrt(variance))

    sns.lineplot(x=x, y=y)
    plt.title("Normal Distribution")
    plt.xlabel("X")
    plt.ylabel("Probability Density")
    plt.xlim([-6, 6])
    plt.show()


mean_slider = FloatSlider(min=-10, max=10, step=0.1, value=0, description="Mean:")
variance_slider = FloatSlider(
    min=0.1, max=10, step=0.1, value=1, description="Variance:"
)

interact(plot_normal_distribution, mean=mean_slider, variance=variance_slider)

interactive(children=(FloatSlider(value=0.0, description='Mean:', max=10.0, min=-10.0), FloatSlider(value=1.0,…

<function __main__.plot_normal_distribution(mean, variance)>

## Central Limit Theorem


In [3]:
def draw_sample_means_exp_distribution(beta):
    """ 
    This code generates an exponential distribution with a lambda parameter 
    of 2 and a size of 100,000. Then, it takes 1,000 samples of size 50 from 
    this distribution, calculates their means, and stores them in a list. 
    Finally, it plots both the original exponential distribution and the 
    distribution of sample means using seaborn's histplot function. The resulting 
    plot of the sample means should resemble a normal distribution, demonstrating 
    the Central Limit Theorem in action.
    """

    # Set seed for reproducibility
    # np.random.seed(42)

    # Define the parameters for the exponential distribution
    lam = 1
    size = 100000

    # Generate the exponential distribution
    data = np.random.exponential(scale=beta, size=size)

    # Draw samples and compute sample means
    sample_size = 50
    num_samples = 1000
    sample_means = []

    for _ in range(num_samples):
        sample = np.random.choice(data, size=sample_size)
        sample_mean = np.mean(sample)
        sample_means.append(sample_mean)


    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

    # Plot the original exponential distribution
    sns.histplot(data, kde=True, color="blue", ax=ax1)
    ax1.set_title("Exponential Distribution")
    ax1.set_xlabel("Values")
    ax1.set_ylabel("Frequency")

    # Plot the distribution of sample means
    sns.histplot(sample_means, kde=True, color="red", ax=ax2)
    ax2.set_title("Distribution of Sample Means")
    ax2.set_xlabel("Sample Means")
    ax2.set_ylabel("Frequency")

    plt.show()

    # Calculate the population variance (if known)
    population_variance = np.var(data, ddof=0)

    # Calculate the sample variance (if population variance is unknown)
    sample_variance = np.var(data, ddof=1)

    # Calculate the variance of the distribution of sample means
    variance_sample_means_pop = population_variance / sample_size
    variance_sample_means_sample = sample_variance / sample_size
    standard_deviation_sample_means_pop = math.sqrt(variance_sample_means_pop)
    standard_deviaation_sample_means_sample = math.sqrt(variance_sample_means_sample)

    print("Variance of sample means (using population variance):", variance_sample_means_pop)
    print("Standard deviation of sample means (using population variance):", standard_deviation_sample_means_pop)

    print("Variance of sample means (using sample variance):", variance_sample_means_sample)
    print("Standard deviation of sample means (using sample variance):", standard_deviaation_sample_means_sample)

beta_slider = IntSlider(
    min=1, max=10, step=1, value=5, description="Beta:"
)

interact(draw_sample_means_exp_distribution, beta=beta_slider)

interactive(children=(IntSlider(value=5, description='Beta:', max=10, min=1), Output()), _dom_classes=('widget…

<function __main__.draw_sample_means_exp_distribution(beta)>

## Binomial distribution - PMF and CDF


In [4]:
def plot_binomial_pmf_cdf(n, p):
    # array of values from 0 to n + 1 that we will use to plot the probability mass function
    x = np.arange(0, n + 1)
    variance = n * p * (1 - p)

    # the probability mass function shows the probability of successes for each value of the x array
    pmf = binom.pmf(x, n, p)

    # the cumulative distribution function shows the probability of getting a number of successes up to a certain value
    cdf = binom.cdf(x, n, p)

    x_axis_min = n / 2 - 100
    x_axis_max = n / 2 + 100
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

    # seaborn plot of the probability mass function with a smoothed curve
    sns.lineplot(x=x, y=pmf, ax=ax1)
    ax1.set_title(
        f"Probability Mass Function for Binomial Distribution (n={n}, p={p}), variance={variance:.0f}"
    )
    ax1.set_xlabel("Number of successes")
    ax1.set_ylabel("Probability of seeing exactly this number of successes")
    ax1.set_xlim([x_axis_min, x_axis_max])
    # ax1.plot(x, pmf, 'bo', ms=8, label='binom pmf')
    ax1.vlines(x, 0, pmf, colors="b", lw=1, alpha=0.5)

    # seaborn plot of the cumulative distribution function
    sns.lineplot(x=x, y=cdf, ax=ax2)
    ax2.set_title(
        f"Cumulative Distribution Function for Binomial Distribution (n={n}, p={p})"
    )
    ax2.set_xlim([x_axis_min, x_axis_max])
    ax2.set_xlabel("Number of successes")
    ax2.set_ylabel("Probability of seeing this number of successes or less")
    # ax2.plot(x, cdf, 'bo', ms=8, label='binom cdf')
    ax2.vlines(x, 0, cdf, colors="b", lw=1, alpha=0.5)

    plt.show()


# plot_binomial_pmf_cdf(1000, 0.5)

sample_size_slider = IntSlider(
    min=200, max=2000, step=100, value=1000, description="Sample Size:"
)
success_rate_slider = FloatSlider(
    min=0.01, max=0.99, step=0.01, value=0.5, description="Success Rate:"
)

interact(plot_binomial_pmf_cdf, n=sample_size_slider, p=success_rate_slider)

interactive(children=(IntSlider(value=1000, description='Sample Size:', max=2000, min=200, step=100), FloatSli…

<function __main__.plot_binomial_pmf_cdf(n, p)>

## Binomial Distribution - Variance

- As sample size increases variance of 'no. of successes' increases
- But variance of the proportion of successes decreases

In [ ]:
def plot_binomial_pmf_cdf_proportion(n, p):
    # create an array of calues from 0 to n + 1
    x = np.arange(0, n + 1)
    x_proportion = x / n

    variance = n * p * (1 - p)
    proportion_variance = p * (1 - p) / n

    # the probability mass function shows the probability of getting a certain number of successes
    pmf = binom.pmf(x, n, p)
    pmf_proportion = pmf / n

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

    sns.lineplot(x=x, y=pmf, ax=ax1)
    ax1.set_title(f"Binomial Distribution (n={n}, p={p}), variance={variance:.0f}")
    ax1.set_xlabel("Number of successes")
    ax1.set_ylabel("Probability of seeing exactly this number of successes")
    ax1.set_xlim([n / 2 - 100, n / 2 + 100])
    ax1.vlines(x, 0, pmf, colors="b", lw=1, alpha=0.5)

    sns.lineplot(x=x_proportion, y=pmf_proportion, ax=ax2)
    plt.title(
        f"Binomial Distribution (n={n}, p={p}), variance={proportion_variance:.5f}"
    )
    plt.xlabel("Proportion of successes")
    ax2.set_ylabel("Probability of seeing exactly this proportion of successes")
    ax2.set_xlim([p - 0.1, p + 0.1])
    ax2.vlines(x_proportion, 0, pmf_proportion, colors="b", lw=1, alpha=0.5)

    plt.show()


sample_size_slider = IntSlider(
    min=200, max=2000, step=100, value=1000, description="Sample Size:"
)
success_rate_slider = FloatSlider(
    min=0.01, max=0.99, step=0.01, value=0.5, description="Success Rate:"
)

interact(plot_binomial_pmf_cdf_proportion, n=sample_size_slider, p=success_rate_slider)

## Significance Level


In [5]:
def plot_binomial_pmf(n, p, alpha, test_sides, x_axis):
    # create an array of values from 0 to n + 1
    x = np.arange(0, n + 1)
    x_proportion = x / n
    proportion_variance = p * (1 - p) / n
    proportion_sd = math.sqrt(proportion_variance)
    x_proportion_sd = (x_proportion - p) / proportion_sd
    v_value = x_proportion if x_axis == "Proportion" else x_proportion_sd

    # the probability mass function shows the probability of getting a certain number of successes
    pmf = binom.pmf(x, n, p)
    pmf_proportion = pmf / n

    plt.figure(figsize=(20, 6))
    ax = sns.lineplot(x=v_value, y=pmf_proportion)
    plt.title(
        f"Binomial Distribution (n={n}, p={p}), variance={proportion_variance:.5f}"
    )
    plt.xlabel("Proportion of successes")
    plt.ylabel("Probability of seeing exactly this proportion of successes")

    xlim_min = p - 4 * proportion_sd if x_axis == "Proportion" else -4
    xlim_max = p + 4 * proportion_sd if x_axis == "Proportion" else 4
    plt.xlim([xlim_min, xlim_max])
    plt.vlines(v_value, 0, pmf_proportion, colors="b", lw=1, alpha=0.5)

    if test_sides == "Two-sided":
        # Show the range of values for a 5% significance level for a two-sided test
        z_score = norm.ppf(1 - alpha / 2)
        lower_bound = (
            p - z_score * np.sqrt(proportion_variance)
            if x_axis == "Proportion"
            else -z_score
        )
        upper_bound = (
            p + z_score * np.sqrt(proportion_variance)
            if x_axis == "Proportion"
            else z_score
        )
        ax.fill_between(
            v_value,
            0,
            pmf_proportion,
            where=((v_value <= lower_bound) | (v_value >= upper_bound)),
            color="r",
            alpha=0.3,
        )
        print("lower bound:", round(lower_bound, 3))
        print("upper bound:", round(upper_bound, 3))

    if test_sides == "One-sided":
        # Show the 5% significance level
        z_score = norm.ppf(1 - alpha)
        upper_bound = (
            p + z_score * np.sqrt(proportion_variance)
            if x_axis == "Proportion"
            else z_score
        )
        ax.fill_between(
            v_value,
            0,
            pmf_proportion,
            where=((v_value >= upper_bound)),
            color="r",
            alpha=0.3,
        )
        print("upper bound:", round(upper_bound, 3))

    plt.show()


alpha_slider = FloatSlider(
    min=0.001, max=0.5, step=0.005, value=0.05, description="Significance Level:"
)
test_sides_buttons = RadioButtons(
    options=["One-sided", "Two-sided"], description="Sides:", value="Two-sided"
)
x_axis_buttons = RadioButtons(
    options=["Proportion", "Standard Deviations"],
    description="X-axis:",
    value="Proportion",
)

interact(
    plot_binomial_pmf,
    n=fixed(1000),
    p=fixed(0.5),
    alpha=alpha_slider,
    test_sides=test_sides_buttons,
    x_axis=x_axis_buttons,
)

interactive(children=(FloatSlider(value=0.05, description='Significance Level:', max=0.5, min=0.001, step=0.00…

<function __main__.plot_binomial_pmf(n, p, alpha, test_sides, x_axis)>